In [ ]:
#!pip install "tensorflow==2.6.0"
#!pip install hexdump

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
print(f"TensorFlow {tf.__version__} devices:\n{tf.config.list_physical_devices()}")

TensorFlow 2.9.2 devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
### ignore TensorFlow INFO messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [4]:
model = tf.keras.Sequential()
model.add(layers.Dense(8, input_dim=2, activation='sigmoid'))
model.add(layers.Dense(1, activation='sigmoid'))

Metal device set to: Apple M1 Max


In [5]:
model.compile(
  loss='binary_crossentropy', 
  optimizer=tf.keras.optimizers.SGD(learning_rate=1), 
  metrics=['accuracy'])

In [6]:
training_data = np.array([[0,0], [0,1], [1,0], [1,1]])
target_data   = np.array([  [0],   [1],   [1],   [0]])

In [7]:
epochs = 500
model.fit(training_data, target_data, epochs=epochs)

Epoch 1/500


2022-06-27 00:33:25.109431: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 262ms/step - loss: 0.7086 - accuracy: 0.5000
Epoch 2/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6943 - accuracy: 0.5000
Epoch 3/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6936 - accuracy: 0.5000
Epoch 4/500
1/1 [==============================] - 0s 7ms/step - loss: 0.6935 - accuracy: 0.5000
Epoch 5/500
1/1 [==============================] - 0s 7ms/step - loss: 0.6935 - accuracy: 0.5000
Epoch 6/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6935 - accuracy: 0.5000
Epoch 7/500
1/1 [==============================] - 0s 7ms/step - loss: 0.6935 - accuracy: 0.5000
Epoch 8/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6934 - accuracy: 0.5000
Epoch 9/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6934 - accuracy: 0.5000
Epoch 10/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6934 - accuracy: 0.5000
Epoch 11/500
1/1 [=====================

1/1 [==============================] - 0s 5ms/step - loss: 0.6888 - accuracy: 0.5000
Epoch 85/500
1/1 [==============================] - 0s 8ms/step - loss: 0.6886 - accuracy: 0.5000
Epoch 86/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6885 - accuracy: 0.5000
Epoch 87/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6884 - accuracy: 0.5000
Epoch 88/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6882 - accuracy: 0.5000
Epoch 89/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6880 - accuracy: 0.5000
Epoch 90/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6879 - accuracy: 0.5000
Epoch 91/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6877 - accuracy: 0.5000
Epoch 92/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6875 - accuracy: 0.5000
Epoch 93/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6873 - accuracy: 0.5000
Epoch 94/500
1/1 [===============

1/1 [==============================] - 0s 5ms/step - loss: 0.6483 - accuracy: 0.7500
Epoch 168/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6473 - accuracy: 0.7500
Epoch 169/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6462 - accuracy: 0.7500
Epoch 170/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6451 - accuracy: 0.7500
Epoch 171/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6440 - accuracy: 0.7500
Epoch 172/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6429 - accuracy: 0.7500
Epoch 173/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6417 - accuracy: 0.7500
Epoch 174/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6405 - accuracy: 0.7500
Epoch 175/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6393 - accuracy: 0.7500
Epoch 176/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6381 - accuracy: 0.7500
Epoch 177/500
1/1 [=====

1/1 [==============================] - 0s 5ms/step - loss: 0.5095 - accuracy: 0.7500
Epoch 250/500
1/1 [==============================] - 0s 5ms/step - loss: 0.5074 - accuracy: 0.7500
Epoch 251/500
1/1 [==============================] - 0s 5ms/step - loss: 0.5053 - accuracy: 0.7500
Epoch 252/500
1/1 [==============================] - 0s 5ms/step - loss: 0.5032 - accuracy: 0.7500
Epoch 253/500
1/1 [==============================] - 0s 5ms/step - loss: 0.5010 - accuracy: 0.7500
Epoch 254/500
1/1 [==============================] - 0s 6ms/step - loss: 0.4989 - accuracy: 0.7500
Epoch 255/500
1/1 [==============================] - 0s 5ms/step - loss: 0.4968 - accuracy: 0.7500
Epoch 256/500
1/1 [==============================] - 0s 5ms/step - loss: 0.4946 - accuracy: 0.7500
Epoch 257/500
1/1 [==============================] - 0s 5ms/step - loss: 0.4925 - accuracy: 0.7500
Epoch 258/500
1/1 [==============================] - 0s 5ms/step - loss: 0.4903 - accuracy: 0.7500
Epoch 259/500
1/1 [=====

1/1 [==============================] - 0s 7ms/step - loss: 0.3166 - accuracy: 1.0000
Epoch 332/500
1/1 [==============================] - 0s 5ms/step - loss: 0.3142 - accuracy: 1.0000
Epoch 333/500
1/1 [==============================] - 0s 6ms/step - loss: 0.3118 - accuracy: 1.0000
Epoch 334/500
1/1 [==============================] - 0s 5ms/step - loss: 0.3094 - accuracy: 1.0000
Epoch 335/500
1/1 [==============================] - 0s 5ms/step - loss: 0.3070 - accuracy: 1.0000
Epoch 336/500
1/1 [==============================] - 0s 7ms/step - loss: 0.3046 - accuracy: 1.0000
Epoch 337/500
1/1 [==============================] - 0s 6ms/step - loss: 0.3022 - accuracy: 1.0000
Epoch 338/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2998 - accuracy: 1.0000
Epoch 339/500
1/1 [==============================] - 0s 5ms/step - loss: 0.2974 - accuracy: 1.0000
Epoch 340/500
1/1 [==============================] - 0s 6ms/step - loss: 0.2951 - accuracy: 1.0000
Epoch 341/500
1/1 [=====

1/1 [==============================] - 0s 5ms/step - loss: 0.1574 - accuracy: 1.0000
Epoch 414/500
1/1 [==============================] - 0s 5ms/step - loss: 0.1561 - accuracy: 1.0000
Epoch 415/500
1/1 [==============================] - 0s 5ms/step - loss: 0.1548 - accuracy: 1.0000
Epoch 416/500
1/1 [==============================] - 0s 5ms/step - loss: 0.1534 - accuracy: 1.0000
Epoch 417/500
1/1 [==============================] - 0s 6ms/step - loss: 0.1521 - accuracy: 1.0000
Epoch 418/500
1/1 [==============================] - 0s 6ms/step - loss: 0.1508 - accuracy: 1.0000
Epoch 419/500
1/1 [==============================] - 0s 5ms/step - loss: 0.1496 - accuracy: 1.0000
Epoch 420/500
1/1 [==============================] - 0s 6ms/step - loss: 0.1483 - accuracy: 1.0000
Epoch 421/500
1/1 [==============================] - 0s 5ms/step - loss: 0.1471 - accuracy: 1.0000
Epoch 422/500
1/1 [==============================] - 0s 6ms/step - loss: 0.1458 - accuracy: 1.0000
Epoch 423/500
1/1 [=====

1/1 [==============================] - 0s 5ms/step - loss: 0.0836 - accuracy: 1.0000
Epoch 496/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0830 - accuracy: 1.0000
Epoch 497/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0825 - accuracy: 1.0000
Epoch 498/500
1/1 [==============================] - 0s 4ms/step - loss: 0.0819 - accuracy: 1.0000
Epoch 499/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0814 - accuracy: 1.0000
Epoch 500/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0809 - accuracy: 1.0000


In [8]:
model.predict(training_data)

1/1 [==============================] - 0s 35ms/step


array([[0.06410554],
       [0.92410344],
       [0.92642903],
       [0.09488381]], dtype=float32)

## C-Code

In [9]:
### tinymlgen - see https://github.com/eloquentarduino/tinymlgen

import re
import hexdump
import tensorflow as tf


def port(model, optimize=False, variable_name='model_data', pretty_print=False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if optimize:
        if isinstance(optimize, bool):
            optimizers = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
        else:
            optimizers = optimize
        converter.optimizations = optimizers
    tflite_model = converter.convert()
    bytes = hexdump.dump(tflite_model).split(' ')
    c_array = ', '.join(['0x%02x' % int(byte, 16) for byte in bytes])
    c = 'const unsigned char %s[] DATA_ALIGN_ATTRIBUTE = {%s};' % (variable_name, c_array)
    if pretty_print:
        c = c.replace('{', '{\n\t').replace('}', '\n}')
        c = re.sub(r'(0x..?, ){12}', lambda x: '%s\n\t' % x.group(0), c)
    c += '\nconst int %s_len = %d;' % (variable_name, len(bytes))
    preamble = '''
// if having troubles with min/max, uncomment the following
// #undef min    
// #undef max

#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif

'''
    return preamble + c

In [10]:
c_code = port(model, optimize=[tf.lite.Optimize.DEFAULT], pretty_print = True)

path = os.getcwd() + "/../pico-tflmicro/examples/xor/"
open(path + "model.cpp", "w").write('#include "model.h"\n' + c_code)

INFO:tensorflow:Assets written to: /var/folders/v9/t6g_nwb51r386g11xp6_3q0w0000gn/T/tmp9apz7ze2/assets


2022-06-27 00:33:29.721840: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-06-27 00:33:29.721854: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.


11883